In [1]:
import sys
sys.path.append("..")

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import accuracy_score
import numpy as np
from dsvit.dataset import BrainTumorDataset
from dsvit.model import DSViTDetector, ViTBaseline, DSViT_NoPreEncoder, DSViT_NoMultiScale
import h5py
from torch.utils.data import Subset


In [ ]:
dataset = BrainTumorDataset(root_dir="/Users/darshdave/Documents/BRAINTUMOR/DATASET/FILES/")

with h5py.File('/Users/darshdave/Documents/BRAINTUMOR/DATASET/cvind.mat', 'r') as f:
    split_labels = np.array(f['cvind']).squeeze().astype(int)

train_ids = np.where(split_labels == 1)[0]
val_ids   = np.where(split_labels == 2)[0]

train_loader = DataLoader(Subset(dataset, train_ids), batch_size=8, shuffle=True)
val_loader   = DataLoader(Subset(dataset, val_ids), batch_size=8)

# subset_ids = list(range(6))
# train_loader = DataLoader(Subset(dataset, subset_ids), batch_size=1, shuffle=True)
# val_loader   = DataLoader(Subset(dataset, subset_ids), batch_size=1)

In [3]:
model = DSViTDetector()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion_cls = nn.CrossEntropyLoss()
criterion_bbox = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [4]:
def evaluate(model, val_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            _, logits = model(x)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend((y - 1).cpu().numpy()) 
    return accuracy_score(all_labels, all_preds)

In [7]:
num_epochs = 10
best_val_acc = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        bbox_pred, class_logits = model(x)
        bbox_gt = torch.tensor([[30.0, 30.0, 40.0, 40.0]] * x.size(0)).to(device)  # Dummy for now

        loss = criterion_cls(class_logits, y - 1) + criterion_bbox(bbox_pred, bbox_gt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "/Users/darshdave/Documents/BRAINTUMOR/DSVIT/model-weight/best_dsvit_detector.pth")
        print("✅ Model saved.")

Epoch 1/10, Train Loss: 200.7608, Val Acc: 0.5000
✅ Model saved.
Epoch 2/10, Train Loss: 196.5875, Val Acc: 0.5000
Epoch 3/10, Train Loss: 194.5485, Val Acc: 0.5000
Epoch 4/10, Train Loss: 191.9150, Val Acc: 0.5000
Epoch 5/10, Train Loss: 189.4788, Val Acc: 0.5000
Epoch 6/10, Train Loss: 187.0057, Val Acc: 0.6667
✅ Model saved.
Epoch 7/10, Train Loss: 184.0167, Val Acc: 0.5000
Epoch 8/10, Train Loss: 180.9163, Val Acc: 1.0000
✅ Model saved.
Epoch 9/10, Train Loss: 177.6254, Val Acc: 1.0000
Epoch 10/10, Train Loss: 174.3467, Val Acc: 1.0000


# ABILATION TRAINING

In [6]:
model = ViTBaseline().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion_cls = nn.CrossEntropyLoss()
criterion_bbox = nn.SmoothL1Loss()

num_epochs = 10
best_val_acc = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0

    for x, y in train_loader:  # use 6-sample mock loader if needed
        x, y = x.to(device), y.to(device)

        bbox_pred, class_logits = model(x)
        bbox_gt = torch.tensor([[30.0, 30.0, 40.0, 40.0]] * x.size(0)).to(device)

        # ✅ shift labels from [1,2,3] → [0,1,2]
        loss = criterion_cls(class_logits, y - 1) + criterion_bbox(bbox_pred, bbox_gt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_acc = evaluate(model, val_loader)  # You already have this function
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "/Users/darshdave/Documents/BRAINTUMOR/DSVIT/model-weight/ViT-Baseline.pth")
        print("✅ ViT-Baseline model saved.")


Epoch 1/10, Train Loss: 209.3240, Val Acc: 0.5000
✅ ViT-Baseline model saved.


: 

In [7]:
model = DSViT_NoPreEncoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion_cls = nn.CrossEntropyLoss()
criterion_bbox = nn.SmoothL1Loss()

num_epochs = 10
best_val_acc = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        bbox_pred, class_logits = model(x)
        bbox_gt = torch.tensor([[30.0, 30.0, 40.0, 40.0]] * x.size(0)).to(device)

        loss = criterion_cls(class_logits, y - 1) + criterion_bbox(bbox_pred, bbox_gt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        path = "/Users/darshdave/Documents/BRAINTUMOR/DSVIT/model-weight/No_Pre-Encoder.pth"
        torch.save(model.state_dict(), path)
        print(f"✅ Saved: {path}")


Epoch 1/10, Loss: 207.0999, Val Acc: 0.5000
✅ Saved: /Users/darshdave/Documents/BRAINTUMOR/DSVIT/model-weight/No_Pre-Encoder.pth
Epoch 2/10, Loss: 200.7487, Val Acc: 0.5000
Epoch 3/10, Loss: 195.8178, Val Acc: 0.5000
Epoch 4/10, Loss: 194.3833, Val Acc: 0.5000
Epoch 5/10, Loss: 191.4619, Val Acc: 0.5000
Epoch 6/10, Loss: 188.5253, Val Acc: 0.6667
✅ Saved: /Users/darshdave/Documents/BRAINTUMOR/DSVIT/model-weight/No_Pre-Encoder.pth
Epoch 7/10, Loss: 185.8213, Val Acc: 0.5000
Epoch 8/10, Loss: 182.8652, Val Acc: 1.0000
✅ Saved: /Users/darshdave/Documents/BRAINTUMOR/DSVIT/model-weight/No_Pre-Encoder.pth
Epoch 9/10, Loss: 179.9650, Val Acc: 0.8333
Epoch 10/10, Loss: 176.7107, Val Acc: 0.5000


In [6]:
model = DSViT_NoMultiScale().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion_cls = nn.CrossEntropyLoss()
criterion_bbox = nn.SmoothL1Loss()

num_epochs = 10
best_val_acc = 0


for epoch in range(num_epochs):
    model.train()
    running_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        bbox_pred, class_logits = model(x)
        bbox_gt = torch.tensor([[30.0, 30.0, 40.0, 40.0]] * x.size(0)).to(device)

        loss = criterion_cls(class_logits, y - 1) + criterion_bbox(bbox_pred, bbox_gt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        path = "/Users/darshdave/Documents/BRAINTUMOR/DSVIT/model-weight/No_Multi-Scale.pth"
        torch.save(model.state_dict(), path)
        print(f"✅ Saved: {path}")


Epoch 1/10, Loss: 211.9580, Val Acc: 0.5000
✅ Saved: /Users/darshdave/Documents/BRAINTUMOR/DSVIT/model-weight/No_Multi-Scale.pth
Epoch 2/10, Loss: 199.9567, Val Acc: 0.5000
Epoch 3/10, Loss: 197.0060, Val Acc: 0.5000
Epoch 4/10, Loss: 194.8164, Val Acc: 0.6667
✅ Saved: /Users/darshdave/Documents/BRAINTUMOR/DSVIT/model-weight/No_Multi-Scale.pth
Epoch 5/10, Loss: 192.1568, Val Acc: 0.5000
Epoch 6/10, Loss: 189.6660, Val Acc: 0.5000
Epoch 7/10, Loss: 187.0334, Val Acc: 0.5000
Epoch 8/10, Loss: 184.5239, Val Acc: 0.5000
Epoch 9/10, Loss: 181.6388, Val Acc: 0.6667
Epoch 10/10, Loss: 178.6108, Val Acc: 0.5000
